# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 0.2850, -1.5543],
         [ 1.3094, -1.7365],
         [ 0.5635,  0.2170],
         [ 1.3448,  0.8380],
         [-1.3990, -0.9523],
         [-0.0397, -2.3836],
         [-1.4629, -0.8347],
         [-1.6063, -0.8349],
         [-2.2457, -1.6658],
         [-1.2781,  1.1153]]),
 tensor([[10.0523],
         [12.7371],
         [ 4.5931],
         [ 4.0547],
         [ 4.6418],
         [12.2328],
         [ 4.1126],
         [ 3.8204],
         [ 5.3829],
         [-2.1435]])]

使用框架的预定义好的层

In [3]:
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

初始化模型参数

In [4]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [5]:
loss = nn.MSELoss()

实例化一个`SGD`实例

In [6]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

训练过程代码与我们从零开始实现时所做的非常相似

In [7]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000349
epoch 2, loss 0.000098
epoch 3, loss 0.000099


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [8]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0013, -0.0002])
b的估计误差： tensor([0.0005])
